In [1]:
%pip install tensorflow==2.15 coremltools


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
import coremltools as ct
import urllib

print("coremltools version:", ct.__version__)
print("tensorflow version:", tf.__version__)

# 加载模型
keras_model = tf.keras.applications.MobileNetV2(
    weights="imagenet", 
    input_shape=(224, 224, 3,),
    classes=1000,
)

# 模型对应的索引文件地址
label_url = 'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt'
class_labels = urllib.request.urlopen(label_url).read().splitlines()
class_labels = class_labels[1:]
assert len(class_labels) == 1000
for i, label in enumerate(class_labels):
    if isinstance(label, bytes):
        class_labels[i] = label.decode("utf8")

# 定义输入
image_input = ct.ImageType(shape=(1, 224, 224, 3), bias=[-1, -1, -1], scale=1/127)

# 设置可预测的标签
classifier_config = ct.ClassifierConfig(class_labels)

# 进行模型转换
model = ct.convert(
    keras_model,
    source="tensorflow",
    inputs=[image_input],
    classifier_config=classifier_config,
)

# 打印模型的输入特征名称
print("模型的输入特征名称:", model.input_description)

# 写入元数据
#model.input_description["input"] = "输入要分类的图片"
#model.output_description["classLabel"] = "最可靠的结果"

# 模型作者
model.author = "TensorFlow转换"

# 许可
model.license = "Please see https://github.com/tensorflow/tensorflow for license information, and https://github.com/tensorflow/models/tree/master/research/slim/nets/mobilenet for the original source of the model."

# 描述
model.short_description = "图片识别模型"

# 版本号
model.version = "1.0"

# 存储模型
model.save("XMobileNetV2.mlpackage")

2024-09-05 12:01:19.869760: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-05 12:01:19.869802: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-05 12:01:19.870661: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-05 12:01:19.877265: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-05 12:01:20.700999: W tensorflow/compiler/tf2

coremltools version: 7.2
tensorflow version: 2.15.0


When both 'convert_to' and 'minimum_deployment_target' not specified, 'convert_to' is set to "mlprogram" and 'minimum_deployment_target' is set to ct.target.iOS15 (which is same as ct.target.macOS12). Note: the model will not run on systems older than iOS15/macOS12/watchOS8/tvOS15. In order to make your model run on older system, please set the 'minimum_deployment_target' to iOS14/iOS13. Details please see the link: https://apple.github.io/coremltools/docs-guides/source/target-conversion-formats.html
2024-09-05 12:01:26.320176: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2024-09-05 12:01:26.320327: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-09-05 12:01:26.322169: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1243 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:c1:00.0, comput

模型的输入特征名称: Features(input_1)


In [5]:
from tensorflow.keras.layers import Layer

# Custom Attention Layer
class Attention(Layer):
    def __init__(self, **kwargs):
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name='attention_weight', shape=(input_shape[-1], 1),
                                 initializer='random_normal', trainable=True)
        self.b = self.add_weight(name='attention_bias', shape=(input_shape[1], 1),
                                 initializer='zeros', trainable=True)
        super(Attention, self).build(input_shape)

    def call(self, x):
        e = tf.nn.tanh(tf.matmul(x, self.W) + self.b)  # Energy
        a = tf.nn.softmax(e, axis=1)  # Attention weights
        output = tf.reduce_sum(x * a, axis=1)  # Weighted sum of input features
        return output

In [11]:
import tensorflow as tf
import coremltools as ct
from tensorflow.keras.models import load_model

# 选择最佳模型
best_model = load_model(f'best_lstm_yolo_model_20240905_1158.h5', custom_objects={'Attention': Attention})

# 转换Core ML 模型
# 定义输入
sequence_input = ct.TensorType(shape=(1, 3, 9))

# 进行模型转换
coreml_model = ct.convert(
    best_model,
    inputs=[sequence_input]
)

# 打印模型的输入和输出特征名称
print("模型的输入特征名称:", coreml_model.input_description)
print("模型的输出特征名称:", coreml_model.output_description)

# 写入元数据
#input_feature_name = list(coreml_model.input_description.keys())[0]
#output_feature_name = list(coreml_model.output_description.keys())[0]

#coreml_model.input_description[input_feature_name] = "输入序列数据"
#coreml_model.output_description[output_feature_name] = "模型输出"

# 模型作者
coreml_model.author = "hello"

# 许可
coreml_model.license = "你的许可信息"

# 描述
coreml_model.short_description = "双向LSTM模型，带有注意力机制"

# 版本号
coreml_model.version = "1.0"

# 存储模型
coreml_output_path = f'best_lstm_yolo_model.mlpackage'
coreml_model.save(coreml_output_path)
print(f"模型成功保存为 Core ML 格式，路径为: {coreml_output_path}")

2024-09-05 12:10:17.157737: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2024-09-05 12:10:17.157853: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-09-05 12:10:17.159513: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1243 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:c1:00.0, compute capability: 8.6
2024-09-05 12:10:18.334874: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2024-09-05 12:10:18.335018: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-09-05 12:10:18.336936: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1243 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:c1:00.0, 

模型的输入特征名称: Features(input_1)
模型的输出特征名称: Features(Identity)
模型成功保存为 Core ML 格式，路径为: best_lstm_yolo_model.mlpackage


In [12]:
import tensorflow as tf
import coremltools as ct
from tensorflow.keras.models import load_model

# 选择最佳模型
best_model = load_model(f'best_lstm_yolo_model_20240905_1158.h5', custom_objects={'Attention': Attention})

# 转换Core ML 模型
# 定义输入
sequence_input = ct.TensorType(shape=(1, 3, 9))

# 进行模型转换
coreml_model = ct.convert(
    best_model, 
    convert_to="neuralnetwork",
    inputs=[sequence_input]
)

# 打印模型的输入和输出特征名称
print("模型的输入特征名称:", coreml_model.input_description)
print("模型的输出特征名称:", coreml_model.output_description)

# 写入元数据
#input_feature_name = list(coreml_model.input_description.keys())[0]
#output_feature_name = list(coreml_model.output_description.keys())[0]

#coreml_model.input_description[input_feature_name] = "输入序列数据"
#coreml_model.output_description[output_feature_name] = "模型输出"

# 模型作者
coreml_model.author = "hello"

# 许可
coreml_model.license = "你的许可信息"

# 描述
coreml_model.short_description = "双向LSTM模型，带有注意力机制"

# 版本号
coreml_model.version = "1.0"

# 存储模型
coreml_output_path = f'best_lstm_yolo_model.mlmodel'
coreml_model.save(coreml_output_path)
print(f"模型成功保存为 Core ML 格式，路径为: {coreml_output_path}")

2024-09-05 12:12:12.466464: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2024-09-05 12:12:12.466579: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-09-05 12:12:12.468281: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1243 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:c1:00.0, compute capability: 8.6
2024-09-05 12:12:13.194744: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2024-09-05 12:12:13.194885: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-09-05 12:12:13.196540: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1243 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:c1:00.0, 

模型的输入特征名称: Features(input_1)
模型的输出特征名称: Features(Identity)
模型成功保存为 Core ML 格式，路径为: best_lstm_yolo_model.mlmodel
